In [ ]:
%run "/Users/hilla.abramov@gmail.com/utils/batch_utils/s3_connector"

In [ ]:
%run "/Users/hilla.abramov@gmail.com/utils/batch_utils/topic_batch_processor"

In [ ]:
%run "/Users/hilla.abramov@gmail.com/utils/batch_utils/config_dicts"

In [ ]:
%run "/Users/hilla.abramov@gmail.com/utils/data_cleaning"

## Extracting the data

In [ ]:
# initiate instance of the S3Connector class (defined in s3_connector notebook in utils folder), 
# using the s3_config_dict (defined in the config_dicts notebook also in the utils folder)
s3_connection = S3Connector(s3_config_dict)
s3_connection.mount_s3_bucket()

# initiate instances of the TopicBatchProcessor class (defined in topic_batch_processor notebook
# in utils folder), per topic, desired topic names can be configured in the config_dicts notebook
# in the utils folder
geo_data_processor = TopicBatchProcessor(s3_connection, topic_names_dict["geolocation"])
pin_data_processor = TopicBatchProcessor(s3_connection, topic_names_dict["pinterest_posts"])
user_data_processor = TopicBatchProcessor(s3_connection, topic_names_dict["users"])

Your S3 bucket has already been mounted under the name of /mnt/user_0a0223c10829_s3_bucket

In [ ]:
# read JSON files from S3 bucket into dataframe for each topic, which is saved as an attribute of
# the respective TopicBatchProcessor instance
for topic_processor in [geo_data_processor, pin_data_processor, user_data_processor]:
    topic_processor.read_json_files_from_s3_bucket_to_df()

## Data Cleaning

In [ ]:
# bespoke cleaning methods imported from notebooks in cleaning_methods folders
pin_data_processor.clean_dataframe(clean_pin_df)
geo_data_processor.clean_dataframe(clean_geo_df)
user_data_processor.clean_dataframe(clean_user_df)

/databricks/spark/python/pyspark/sql/dataframe.py:2374: UserWarning: to_replace is a dict and value is not None. value will be ignored.
 warnings.warn("to_replace is a dict and value is not None. value will be ignored.")

## Run queries on updated master dataset (outputs written to Parquet files)

In [ ]:
# to enable sql queries through spark, creating temporary views of each dataframe in
# the spark session
import re

for topic_processor in [geo_data_processor, pin_data_processor, user_data_processor]:
    # creates table names "hilla_geo", "hilla_pin" and "hilla_user" to avoid interference
    # with other session users
    sql_table_name = re.sub(r'^\w+\.', 'hilla_', topic_processor._topic_name)
    topic_processor._cleaned_batch_data.createOrReplaceTempView(sql_table_name)

In [ ]:
%run "/Users/hilla.abramov@gmail.com/utils/batch_utils/data_queries"